In [1]:
import numpy as np
from keras.layers import Dense, Activation
from keras.layers.recurrent import SimpleRNN, LSTM, GRU
from keras.models import Sequential

Using TensorFlow backend.


In [2]:
# построчное чтение из примера с текстом 
with open("pochtamt.txt", 'rb') as _in:
    lines = []
    for line in _in:
        line = line.strip().lower().decode("utf-8","ignore")
        if len(line) == 0:
            continue
        lines.append(line)
text = " ".join(lines)
chars = set([c for c in text])
nb_chars = len(chars)

# создание индекса символов и reverse mapping чтобы передвигаться между значениями numerical
# ID and a specific character. The numerical ID will correspond to a column
# ID и определенный символ. Numerical ID будет соответсвовать колонке
# число при использовании one-hot кодировки для представление входов символов
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

# для удобства выберете фиксированную длину последовательность 10 символов 
SEQLEN, STEP = 50, 1
input_chars, label_chars = [], []

# конвертация data в серии разных SEQLEN-length субпоследовательностей
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])


# Вычисление one-hot encoding входных последовательностей X и следующего символа (the label) y

X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1

In [6]:
# установка ряда метапамертров  для нейронной сети и процесса тренировки
BATCH_SIZE, HIDDEN_SIZE = 128, 128
NUM_ITERATIONS = 100
NUM_EPOCHS_PER_ITERATION = 2
NUM_PREDS_PER_EPOCH = 100

In [7]:
model = Sequential()
model.add(
    LSTM(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=False,
        input_shape=(SEQLEN, nb_chars),
        unroll=True
    )
)
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [8]:
# выполнение серий тренировочных и демонстрационных итераций 
for iteration in range(NUM_ITERATIONS):

    # для каждой итерации запуск передачи данных в модель 
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    # для числа шагов предсказаний использование текущей тренируемой модели 
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):

        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру 
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred
print()

Итерация #: 0
Epoch 1/2
223660/223660 [==============================] - 135s 602us/step - loss: 2.7170
Epoch 2/2
223660/223660 [==============================] - 137s 613us/step - loss: 2.3334
Генерация из посева:  имел, на что согласился. Зона 9 была самым здоров
 имел, на что согласился. Зона 9 была самым здоровал с польше подаль в подел в подель в подел в подель в подел в подель в подел в подель в подел в под==================================================
Итерация #: 1
Epoch 1/2
223660/223660 [==============================] - 138s 616us/step - loss: 2.1783
Epoch 2/2
223660/223660 [==============================] - 138s 617us/step - loss: 2.0707
Генерация из посева: . Потом я поджег почтамт. Меня отправили сортирова
. Потом я поджег почтамт. Меня отправили сортировали в польшение подного подомал он постоли поледной поледной поледной поледной поледной поледной поле==================================================
Итерация #: 2
Epoch 1/2
223660/223660 [===========================

ми разбухали. Штатные доставщики начинали бюллетенить в постель и поднялся и посмотрел на меня с ним, и он на меня от машина на старом дверьшиется с с==================================================
Итерация #: 35
Epoch 1/2
223660/223660 [==============================] - 133s 596us/step - loss: 1.2437
Epoch 2/2
223660/223660 [==============================] - 133s 594us/step - loss: 1.2412
Генерация из посева: Если б елка обожгла ей сиськи или сраку, она б ее 
Если б елка обожгла ей сиськи или сраку, она б ее до женные почтамта, в машину. Я вышел в нее страницы подняли на ногом и спросила на меня своего дома==================================================
Итерация #: 36
Epoch 1/2
223660/223660 [==============================] - 133s 594us/step - loss: 1.2418
Epoch 2/2
223660/223660 [==============================] - 132s 592us/step - loss: 1.2394
Генерация из посева: ском Участке, и везде. Даже сменщики сказывались б
ском Участке, и везде. Даже сменщики сказывались будет в сортиру

ик записал мне опоздание. Я никогда больше не видел я начал ходить в своему времени, как он открывал месте. Все ужо стояло столкнули к работе. Я вышел==================================================
Итерация #: 70
Epoch 1/2
223660/223660 [==============================] - 135s 602us/step - loss: 1.1792
Epoch 2/2
223660/223660 [==============================] - 133s 595us/step - loss: 1.1790
Генерация из посева: их маленьких секретов не выдавали. Это было их еди
их маленьких секретов не выдавали. Это было их единструсно. Я подошел к планой назвом полном на столу. Я поднял глядить должение почтамтам и подносы с==================================================
Итерация #: 71
Epoch 1/2
223660/223660 [==============================] - 134s 600us/step - loss: 1.1778
Epoch 2/2
223660/223660 [==============================] - 134s 599us/step - loss: 1.1779
Генерация из посева:  отрезка, и тут они наступят ей на пятки. Стандарт
 отрезка, и тут они наступят ей на пятки. Стандартие водить и ст

Увеличение числа эпох и итераций улучшили выдачу, но до больших результатов конечно необходимо большее количество циклов и железо мощнее. Так же стало очевидно что необходима тчательная предобработка текста, удаление всех пустых абзаций и не имеющей значение информации. Это было сделано, и как вижу по выдаче еще стоило удалить знаки препинания. 